In [1]:
numeric_stats = ['derived_fg', 'derived_ft', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
fantasy_stats = ['FG', 'FGA', 'FTA', 'FT', 'TRB', 'AST', 'STL', 'BLK', 'TOV', '3P', 'PTS']
negative_stats = ['FTA', 'TOV', 'FGA']

In [2]:
import unidecode

def name_transform(name):
    name = name.split('\\')[0].strip()
    name = unidecode.unidecode(name)
    transforms = {}
    transforms['Marcus Morris'] = 'Marcus Morris Sr.'
    transforms['Otto Porter'] = 'Otto Porter Jr.'
    if name in transforms:
        return transforms[name]
    return name
name_transform('Otto Porter')

'Otto Porter Jr.'

In [3]:
# Import stats CSV
import pandas as pd

arr = pd.read_csv('current_season.csv')
arr['Player'] = arr['Player'].transform(name_transform)
arr = arr.drop(['Rk', 'GS', 'Age', 'Pos', 'FG%', 'FT%', '2P%', 'eFG%', '3PA', '3P%', '2P', '2PA', 'Tm'], axis=1)
arr = arr.groupby(arr['Player']).sum()
arr['FT%'] = arr['FT'] * 100 / arr['FTA']
arr['FG%'] = arr['FG'] * 100 / arr['FGA']
arr['derived_fg'] = arr['FG'] * (arr['FG%'] - arr['FG%'].mean())
arr['derived_ft'] = arr['FT'] * (arr['FT%'] - arr['FT%'].mean())
arr[['TRB', 'AST', 'STL', 'BLK', 'TOV', '3P', 'PTS']] = arr[['TRB', 'AST', 'STL', 'BLK', 'TOV', '3P', 'PTS']].div(arr['G'], axis=0)

In [4]:
transform_functions = {}
for col in arr.iteritems():
    stat = col[0]
    if stat in numeric_stats:
        std_dev = arr[stat].std()
        mean = arr[stat].mean()
        if stat in negative_stats:
            std_dev=std_dev*-1
        transform_functions[stat] = lambda val,mean=mean,std_dev=std_dev: (val-mean)*1000/std_dev
    else:
        transform_functions[stat] = lambda val: val
arr = arr.dropna()
z_scores = arr.transform(transform_functions)

In [5]:
z_scores['zach_points'] = z_scores[fantasy_stats].sum(axis=1)
stats = arr[fantasy_stats+['FT%', 'FG%']].copy()
stats['total_production'] = z_scores['zach_points']

In [6]:
print(stats.to_json(orient='index'))

{"Aaron Gordon":{"FG":60,"FGA":126,"FTA":46,"FT":27,"TRB":6.1818181818,"AST":2.8181818182,"STL":0.8181818182,"BLK":0.6363636364,"TOV":2.0,"3P":1.6363636364,"PTS":15.0,"FT%":58.6956521739,"FG%":47.619047619,"total_production":2367.5986588121},"Aaron Holiday":{"FG":30,"FGA":82,"FTA":6,"FT":5,"TRB":1.4166666667,"AST":2.5,"STL":1.0,"BLK":0.1666666667,"TOV":0.6666666667,"3P":0.8333333333,"PTS":6.25,"FT%":83.3333333333,"FG%":36.5853658537,"total_production":-600.3438294506},"Al Horford":{"FG":41,"FGA":98,"FTA":4,"FT":2,"TRB":7.1111111111,"AST":2.4444444444,"STL":0.3333333333,"BLK":0.7777777778,"TOV":0.8888888889,"3P":1.7777777778,"PTS":11.1111111111,"FT%":50.0,"FG%":41.8367346939,"total_production":2952.6586823922},"Alec Burks":{"FG":17,"FGA":32,"FTA":20,"FT":18,"TRB":3.6666666667,"AST":3.6666666667,"STL":1.0,"BLK":0.6666666667,"TOV":2.3333333333,"3P":3.3333333333,"PTS":20.6666666667,"FT%":90.0,"FG%":53.125,"total_production":5294.8792037017},"Alex Caruso":{"FG":18,"FGA":36,"FTA":2,"FT":1,"T